In [1]:
import os
import nibabel as nib
import numpy as np
from scipy.ndimage.interpolation import zoom
import scipy as sp
from tqdm.notebook import tqdm_notebook

import keras


def load_nifti(file_path, mask=None, z_factor=None, remove_nan=False):
    """Load a 3D array from a NIFTI file."""
    img = nib.load(file_path)
    struct_arr = np.array(img.get_data())

    if remove_nan:
        struct_arr = np.nan_to_num(struct_arr)
    if mask is not None:
        struct_arr *= mask
    if z_factor is not None:
        struct_arr = np.around(zoom(struct_arr, z_factor), 0)

    return struct_arr


def save_nifti(file_path, struct_arr):
    """Save a 3D array to a NIFTI file."""
    img = nib.Nifti1Image(struct_arr, np.eye(4))
    nib.save(img, file_path)
    




In [25]:
def prepareData(use_smooth = False, runningOnServer = False):
    rootDir = '/home/eshan/Downloads/PD DATA/' if not runningOnServer else '/mnt/hdd1/lxc-hdd1/tahjid/PD Data/'
    grayMatter, whiteMatter, fullScan, labels, patientList, patientNumbers, patientIds = [], [], [], [], [], [], []
    labelMap = dict(Control=0, PD=1)
    typeMap = dict(FullScan=0, GrayMatter=1, WhiteMatter=2)
    fullScanPath = rootDir + 'FinalData/'
    wmgmpath = rootDir + 'FinalDataWMGM/' if not use_smooth else rootDir + 'FinalDataWMGMSmooth/'
    prefix = 'mwp' if not use_smooth else 'smwp'
    ext = '.nii'
    for i in ['Control', 'PD']:
        path = fullScanPath + i + '/'
        listOfFiles = [f for f in os.listdir(path) if f.endswith(ext)]
        for file in listOfFiles:
            patientNumbers.append(file[:4])
        path = wmgmpath + i + '/'
        listOfFiles = [f for f in os.listdir(path) if f.endswith(ext)]
        for file in listOfFiles:
            filename = file[4:8] if not use_smooth else file[5:9]
            if filename not in patientNumbers:
                continue
            if not use_smooth:
                patientList.append([i, file[4:8]])
            else:
                patientList.append([i, file[5:9]])

    for i in patientList:
        path = fullScanPath + i[0] + '/'
        patientIds.append(i[1])
        fullScan.append(os.path.join(path + i[1] + ext))
        path = wmgmpath + i[0] + '/'
        grayMatter.append(os.path.join(path + prefix + str(typeMap['GrayMatter']) + i[1] + ext))
        whiteMatter.append(os.path.join(path + prefix + str(typeMap['WhiteMatter']) + i[1] + ext))
        labels.append(labelMap[i[0]])
    return fullScan, grayMatter, whiteMatter, labels, patientIds
a,b,c,d,e =prepareData()

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
    

In [26]:
a[0]

'/home/eshan/Downloads/PD DATA/FinalData/Control/5007.nii'

In [27]:
b[0]

'/home/eshan/Downloads/PD DATA/FinalDataWMGM/Control/mwp15007.nii'

In [28]:
c[0]

'/home/eshan/Downloads/PD DATA/FinalDataWMGM/Control/mwp25007.nii'

In [29]:
d[0]

0

In [30]:
e[0]

'5007'